### steps
1. Create an sample SQL database and table for testing cases
2. performing SQL window on database to solve Consecute number problem

In [45]:
from mysql.connector import MySQLConnection, Error
from python_mysql_dbconfig import read_db_config # Read database configuration file and return a dictionary database object

In [44]:
def create_msql_database(name_database):
    """Function create MYSQL database"""
    mydb = mysql.connector.connect(host='localhost',
                                      user='root',
                                      password='root')
    
    mycursor = mydb.cursor()
    mycursor.execute("SHOW DATABASES")
    databases = [x[0] for x in mycursor]
    if name_database not in databases:
        mycursor.execute(f"CREATE DATABASE IF NOT EXISTS {name_database}")
    # Check database


In [47]:
def insert_msql_table(query_table, table_values):
    """execute query to create table and insert values to table
    :query_table:: to create scheme table
    :table_values:: to insert values to table
    """

    conn = None
    try:
        db_config = read_db_config()
        print('Connecting to MYSQL database...')
        conn = MySQLConnection(**db_config)

        cursor= conn.cursor()
        cursor.executemany(query_table, table_values)

        conn.commit()

    except Error as e:
        print("Error: ", e)

    finally:
        cursor.close()
        conn.close()

    mycursor = conn.cursor()


